In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import nltk

# Ensure you have downloaded the necessary NLTK data
nltk.download('punkt')

# Load the tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
# Define the path to your CSV file
csv_file_path = 'legal project/labelled_jan1950-2024.csv'  # Update this to your CSV file's location

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

In [ ]:
df.head()

,title,link,fact,Result,Label
0,"Kapildeo Singh vs The King on 24 January, 1950",https://indiankanoon.org/docfragment/1878796/?...,1.Thisisanappealbyspecialleaveagainstanorderof...,Rejected,0
1,The State Of Bombay vs Atma Ram Sridhar Vaidya...,https://indiankanoon.org/docfragment/1382411/?...,CRIMINALAPPELLATEJURISDICTIONCaseNo.22of1950. ...,Rejected,0
2,The State Of West Bengal vs Anwar Ali Sarkar o...,https://indiankanoon.org/docfragment/184660633...,1.ThisisanappealbytheStateofWestBengalfromajud...,Rejected,0
3,Nalinakhya Bysack vs Shyam Sunder Haldar And O...,https://indiankanoon.org/docfragment/553711/?f...,CIVILAPPELLATEJURISDICTIONCivilAppealNo.96of19...,Accepted,1
4,K.S. Rashid And Son vs The Income-Tax Investig...,https://indiankanoon.org/docfragment/941160/?f...,CIVILAPPELLATEJURISDICTIONCIVILAPPEALSNos. 118...,Accepted,1


In [ ]:
import re

# Initialize a list to store preprocessed text
preprocessed_texts = []
labels = []

# Define regular expressions for different outcomes
outcomes = [
    ("executivediscertion", 1),
    ("sentenced", 1),
    ("appealaccepted\.", 1),
    ("appealrejected\.", 0),
    ("appealdisposed\.", 0),
    ("appealdismissed\.", 0),
    ("appealallowed\.", 1),
    ("petitionaccepted\.", 1),
    ("petitionrejected\.", 0),
    ("petitiondisposed\.", 0),
    ("petitiondismissed\.", 0),
    ("petitionallowed\.", 1),
    ("appealsaccepted\.", 1),
    ("appealsrejected\.", 0),
    ("appealsdisposed\.", 0),
    ("appealsdismissed\.", 0),
    ("appealsallowed\.", 1),
    ("petitionsaccepted\.", 1),
    ("petitionsrejected\.", 0),
    ("petitionsdisposed\.", 0),
    ("petitionsdismissed\.", 0),
    ("petitionsallowed\.", 1),
    ("appeal.*accepted", 1),
    ("appeal.*rejected", 0),
    ("appeal.*disposed", 0),
    ("appeal.*dismissed", 0),
    ("appeal.*allowed", 1),
    ("petition.*accepted", 1),
    ("petition.*rejected", 0),
    ("petition.*disposed", 0),
    ("petition.*dismissed", 0),
    ("petition.*allowed", 1),
    ("appeals.*accepted", 1),
    ("appeals.*rejected", 0),
    ("appeals.*disposed", 0),
    ("appeals.*dismissed", 0),
    ("appeals.*allowed", 1),
    ("petitions.*accepted", 1),
    ("petitions.*rejected", 0),
    ("petitions.*disposed", 0),
    ("petitions.*dismissed", 0),
    ("petitions.*allowed", 1),
    ("appeal.*apply", 1),
    ("appeals.*apply", 1),
    ("petitions.*apply", 1),
    ("petition.*apply", 1),
]

# Process each row in the DataFrame
for index, row in df.iterrows():
    text = row['fact'].lower()  # Assuming the column name is 'fact'
    textf = text[-100:]  # Last 100 characters for label finding

    processed = False
    for outcome, label in outcomes:
        if re.search(outcome, text):
            preprocessed_texts.append(outcome)
            labels.append(label)
            processed = True
            break  # Stop checking after the first match

    if not processed:
        # Handle rows that don't match any outcome by adding the original text and a default label
        preprocessed_texts.append(row["Result"])
        labels.append(row['Label'])  # Use None or a specific value that indicates unprocessed/uncategorized

# Add the preprocessed text and labels as new columns in the DataFrame
df['Preprocessedoutcome'] = preprocessed_texts
df['Label2'] = labels

In [ ]:
df.head()

,title,link,fact,Result,Label,Preprocessedoutcome,Label2
0,"Kapildeo Singh vs The King on 24 January, 1950",https://indiankanoon.org/docfragment/1878796/?...,1.Thisisanappealbyspecialleaveagainstanorderof...,Rejected,0,sentenced,1
1,The State Of Bombay vs Atma Ram Sridhar Vaidya...,https://indiankanoon.org/docfragment/1382411/?...,CRIMINALAPPELLATEJURISDICTIONCaseNo.22of1950. ...,Rejected,0,appealallowed\.,1
2,The State Of West Bengal vs Anwar Ali Sarkar o...,https://indiankanoon.org/docfragment/184660633...,1.ThisisanappealbytheStateofWestBengalfromajud...,Rejected,0,sentenced,1
3,Nalinakhya Bysack vs Shyam Sunder Haldar And O...,https://indiankanoon.org/docfragment/553711/?f...,CIVILAPPELLATEJURISDICTIONCivilAppealNo.96of19...,Accepted,1,appealallowed\.,1
4,K.S. Rashid And Son vs The Income-Tax Investig...,https://indiankanoon.org/docfragment/941160/?f...,CIVILAPPELLATEJURISDICTIONCIVILAPPEALSNos. 118...,Accepted,1,appealsdismissed\.,0


In [ ]:
df['Label2'].value_counts()

1    50
0    25
Name: Label2, dtype: int64

In [ ]:
df['Preprocessedoutcome'].value_counts()

appeal.*accepted       24
sentenced              15
appeal.*rejected        6
appeal.*dismissed       6
appeal.*allowed         6
appealallowed\.         4
appealsdismissed\.      4
appealdismissed\.       4
appeal.*disposed        3
petitiondismissed\.     1
petition.*rejected      1
petition.*allowed       1
Name: Preprocessedoutcome, dtype: int64

In [ ]:
df['Preprocessedoutcome'].isnull().sum()

0

In [ ]:
# Extract rows where 'Preprocessedoutcome' is None
data = df[df['Preprocessedoutcome'].isnull()]

In [ ]:
data.head()

,title,link,fact,Result,Label,Preprocessedoutcome,Label2


In [ ]:
# Save the updated DataFrame back to the CSV file or a new file
output_csv_file_path = 'path_to_your_output_csv_file.csv'  # Update this to where you want to save the updated CSV
df.to_csv(output_csv_file_path, index=False)

print("CSV file updated with preprocessed data.")